In [50]:
from functools import reduce
from itertools import chain, repeat
from nltk import FreqDist

from lib.util import load_json

raw_tokens = chain(
    load_json('../tmp/raw_tokens.json'),
    load_json('../tmp/neutral_corpus/tokens.json'),
)

dist = FreqDist(raw_tokens)

token_count = reduce(lambda x, y: x+y, dist.values(), 0)

probs = {
    k: v/token_count
    for k, v
    in dist.items()
}

def est_phrase_probability(phrase, probs):
    return reduce(lambda x, y: x * probs[y], phrase, 1)

# phrase = 'an athletic opportunity or suffers any direct or indirect harm result of'.split(' ')
# prob_thes = est_phrase_probability(repeat('the', len(phrase)), probs)
# prob_phrase = est_phrase_probability(phrase, probs)
# print(prob_thes, prob_phrase, prob_thes/prob_phrase, prob_phrase/prob_thes)

# phrase = 'law of utah chapter renumbered from last amend by law of utah'.split(' ')
# prob_thes = est_phrase_probability(repeat('the', len(phrase)), probs)
# prob_phrase = est_phrase_probability(phrase, probs)
# print(prob_thes, prob_phrase, prob_thes/prob_phrase, prob_phrase/prob_thes)

ngrams = load_json('../tmp/grams-24.json')
tagged_ngrams = {
    ngram: (
        len(occurrences), 
        (
            est_phrase_probability(repeat('the', len(ngram.split(' '))), probs)
            / est_phrase_probability(ngram.split(' '), probs)
        ),
    )
    for ngram, occurrences
    in ngrams.items()
}

sorted(tagged_ngrams.items(), key=lambda x: x[1][1], reverse=True)

[('biologically male patient such a augmentation mammoplasty facial feminization surgery liposuction lipofilling voice surgery thyroid cartilage reduction gluteal augmentation hair reconstruction or various aesthetic',
  (21, 2.2483332290270653e+67)),
 ('male patient such a augmentation mammoplasty facial feminization surgery liposuction lipofilling voice surgery thyroid cartilage reduction gluteal augmentation hair reconstruction or various aesthetic procedure',
  (21, 1.343914854184191e+66)),
 ('surgical procedure for biologically male patient such a augmentation mammoplasty facial feminization surgery liposuction lipofilling voice surgery thyroid cartilage reduction gluteal augmentation hair reconstruction',
  (21, 7.133681125121759e+65)),
 ('for biologically male patient such a augmentation mammoplasty facial feminization surgery liposuction lipofilling voice surgery thyroid cartilage reduction gluteal augmentation hair reconstruction or various',
  (21, 5.299039647464375e+64)),
 (

In [13]:
from nltk import ngrams, FreqDist
from pprint import pprint

from lib.tasks import tokenize_corpus
from lib.util import load_json

stopwords = set(load_json('../artifacts/legal_stopwords.json'))

#raw_tokens = list(tokenize_corpus('../bills/*'))

import json

# with open('../tmp/raw_tokens.json', 'w') as f:
#     json.dump(raw_tokens, f)

with open('../tmp/raw_tokens.json', 'r') as f:
    raw_tokens = json.load(f)

filtered_tokens = [
    token
    for token
    in raw_tokens
    if len(token) > 1
]

for i in range(2,51):
    grams = ngrams(filtered_tokens, i)
    dist = FreqDist(grams)
    
    serializable = {
        ' '.join(k): v
        for k, v
        in dist.items()
        if v > 20
    }

    with open(f'../tmp/ngrams-{str(i).zfill(2)}.json', 'w') as f:
        json.dump(serializable, f, indent=2)
        
# tengrams = ngrams(list(filtered_tokens), 10)
# dist = FreqDist(tengrams)

# # pprint(sorted([
# #     (k, v)
# #     for k, v in dist.items()
# #     if v > 60
# # ], key=lambda x: x[1], reverse=True))


# serializable = {
#     ' '.join(k): v
#     for k, v
#     in dist.items()
#     if v > 20
# }

# with open('../tmp/tengrams.json', 'w') as f:
#     json.dump(serializable, f, indent=2)

In [ ]:
# 1700313

import pandas as pd

summaries = pd.read_json('../tmp/neutral_corpus/summaries.json')
summaries.loc[summaries.legiscan_bill_id == 1700313]

In [ ]:
import pdfplumber
from pprint import pprint
from PyPDF2 import PdfReader

def extract_pdf_tokens_pypdf2(file_path: str) -> str:
    reader = PdfReader(file_path)
    return reader.pages[0].extract_text()

def extract_pdf_tokens_pdfplumber(file_path: str) -> str:
    with pdfplumber.open(file_path) as pdf:
        return pdf.pages[0].extract_text()

pprint(extract_pdf_tokens_pdfplumber('../archive/bills/OK_SB973.pdf'))

In [ ]:
from glob import glob
import re

def find_hyphenate(text: str):
    return re.subn(r'(\w)+-\s*\n\s*(\w+)', r'\g<1>\g<2>\n', text, re.MULTILINE)

sample = 'the quick the \n brown fox jumps o- \n ver the \n lazy dog'
print(find_hyphenate(sample))

re.search(r'(\w)+-\s*\n\s+(\w+)', sample).groups()
# glob('../tmp/neutral_corpus/bills/*.pdf')

In [ ]:
import sys
sys.path.append('..')

from bs4 import BeautifulSoup as Soup
from collections import namedtuple
import json
from retrieval.legiscan import legiscan_auth

Dataset = namedtuple("Dataset", "state year session modified exported json csv")

@legiscan_auth
def enumerate_datasets(session):
    return session.get('https://legiscan.com/datasets').text

soup = Soup(enumerate_datasets())
dataset_table = soup.find(id='gaits-datasets')
table_data = [
    Dataset(*(
        cell.text if len(cell.find_all('a'))<1 else cell.find_all('a')[0].attrs['href']
        for cell 
        in row.find_all('td')))
    for row
    in dataset_table.find_all('tbody')[0].find_all('tr')
]

len([item.json for item in table_data if '2023' in item.session])


In [ ]:
import json

with open('../datasets/geography.json', 'r') as f:
    geography = json.load(f)
    
pairs = [
('ALABAMA', 'AL'),
('ALASKA', 'AK'),
('AMERICAN SAMOA', 'AS'),
('ARIZONA', 'AZ'),
('ARKANSAS', 'AR'),
('CALIFORNIA', 'CA'),
('COLORADO', 'CO'),
('CONNECTICUT', 'CT'),
('DELAWARE', 'DE'),
('DISTRICT OF COLUMBIA', 'DC'),
('FLORIDA', 'FL'),
('GEORGIA', 'GA'),
('GUAM', 'GU'),
('HAWAII', 'HI'),
('IDAHO', 'ID'),
('ILLINOIS', 'IL'),
('INDIANA', 'IN'),
('IOWA', 'IA'),
('KANSAS', 'KS'),
('KENTUCKY', 'KY'),
('LOUISIANA', 'LA'),
('MAINE', 'ME'),
('MARYLAND', 'MD'),
('MASSACHUSETTS', 'MA'),
('MICHIGAN', 'MI'),
('MINNESOTA', 'MN'),
('MISSISSIPPI', 'MS'),
('MISSOURI', 'MO'),
('MONTANA', 'MT'),
('NEBRASKA', 'NE'),
('NEVADA', 'NV'),
('NEW HAMPSHIRE', 'NH'),
('NEW JERSEY', 'NJ'),
('NEW MEXICO', 'NM'),
('NEW YORK', 'NY'),
('NORTH CAROLINA', 'NC'),
('NORTH DAKOTA', 'ND'),
('NORTHERN MARIANA IS', 'MP'),
('OHIO', 'OH'),
('OKLAHOMA', 'OK'),
('OREGON', 'OR'),
('PENNSYLVANIA', 'PA'),
('PUERTO RICO', 'PR'),
('RHODE ISLAND', 'RI'),
('SOUTH CAROLINA', 'SC'),
('SOUTH DAKOTA', 'SD'),
('TENNESSEE', 'TN'),
('TEXAS', 'TX'),
('UNITED STATES', 'US'),
('UTAH', 'UT'),
('VERMONT', 'VT'),
('VIRGINIA', 'VA'),
('VIRGIN ISLANDS', 'VI'),
('WASHINGTON', 'WA'),
('WEST VIRGINIA', 'WV'),
('WISCONSIN', 'WI'),
('WYOMING', 'WY'),
]

def fix_case(state: str):
    return ' '.join(word.capitalize() for word in state.split(' '))

geography['state_abbreviations'] = {fix_case(tup[0]): tup[1] for tup in pairs}
geography['state_names'] = {tup[1]: fix_case(tup[0]) for tup in pairs}

with open('../datasets/geography.json', 'w') as f:
    json.dump(geography, f, indent=2)

In [ ]:
import glob
from operator import itemgetter
import json

metas =  glob.glob('../tmp/legiscan/*meta*.json')
def get_meta(meta_name):
    with open(meta_name, 'r') as f:
        return json.load(f)
    
def summarize(meta):
    bill = meta['bill']
    state, bill_id, status_date, legiscan_bill_id = itemgetter('state', 'bill_number', 'status_date', 'bill_id')(bill)
    
    return {
        'state': state,
        'bill_id': bill_id,
        'status_date': status_date,
        'legiscan_bill_id': legiscan_bill_id,
    }

[summarize(get_meta(meta)) for meta in metas][0:20]

In [ ]:
from bs4 import BeautifulSoup as Soup
from itertools import chain, islice, takewhile
import json
import re
import requests
import string
import time

TRACKER_URL = 'https://www.equalitytexas.org/legislature/legislative-bill-tracker-2023'
OUTPUT_PATH = '../datasets/equalitytexas.json'

def extract_row(row):
    cells = row.find_all('td')
    d = re.search(r'\d{2}/\d{2}/\d{4}', cells[3].text)
    return {
        'state': 'TX',
        'bill_id': cells[0].text,
        'sponsors': [sponsor for sponsor in cells[1].text.split(' ') if sponsor not in string.punctuation],
        'description': cells[2].text,
        'status_date': d.group(0) if d else '',
    }

start_time = time.time()

page = requests.get(TRACKER_URL)

soup = Soup(page.content, 'html.parser')
bad_bills = soup.find(id='bad-bills')

bill_tables = islice(bad_bills.parent.parent.find_all('table'), 1, None)
relevant_rows = chain.from_iterable(
    (row for row in tbl.find_all('tr') if not row.find('th'))
    for tbl
    in bill_tables
)

dataset = list((takewhile(lambda r: r['bill_id'] != '#N/A', (extract_row(row) for row in relevant_rows))))

with open(OUTPUT_PATH, 'w') as f:
    json.dump(dataset, f, indent=2)

end_time = time.time()

print(f'Data "equalitytexas.json" refreshed with {len(dataset)} items ({(end_time-start_time):.2f}s elapsed)')

In [ ]:
from bs4 import BeautifulSoup as Soup
from itertools import chain
import json
from pprint import pprint
from pyjsparser import parse
import requests
from urllib.parse import urljoin

URL = 'https://tracktranslegislation.com'
page = requests.get(URL)
soup = Soup(page.content, 'html.parser')
script_tags = soup.find_all('script')
sources = [source_tag.attrs['src'] for source_tag in soup.find_all('script') if source_tag.has_attr('src')]
quarry = urljoin(URL, next(source for source in sources if 'chunks/70-' in source))
script_contents = requests.get(quarry).text

def find_in_graph(subgraph):
    results = []
    items = subgraph.values() if isinstance(subgraph, dict) else subgraph
    local_results = (item for item in items if isinstance(item, str) and len(item) > 10000)
    return (chain.from_iterable([local_results, *(find_in_graph(item) for item in items if isinstance(item, dict) or isinstance(item,list))]))

parsed = parse(script_contents)
candidates = find_graph(parsed)
jsonstr = next(candidates)
len(json.loads(jsonstr))


In [ ]:
t1 = ('a', 3)
t2 = (*t1, 'b')
t2

In [ ]:
from itertools import chain, islice
import json

mapper = {}
with open('resolver_map.json', 'r') as f:
    mapper = json.load(f)

list(islice(chain.from_iterable(
    ((state, k, v) for k, v in m['bills'].items())
    for state, m 
    in mapper.items()
), 10))


In [ ]:
from mergedeep import merge

a = {
    'foo': {
        'bar': 'baz',
        'glarch': 'glarch',
    },
    'bar': 3,
    'arr': [3]
}

b = {
    'foo': {
        'bar': 'bar',
    },
    'baz': 'baz',
    'bar': {
        'blah': 'blah'
    },
    'arr': [4]
}

c = {
    'foo': { }
}

dummy = {}
merge(dummy, a, b, c)
pprint(dummy)


In [ ]:
import json
from legiscan import legiscan_api
import requests

@legiscan_api
def do_search(state: str, term: str, api_key: str):
    short_url = 'https://api.legiscan.com/'
    assemble_url = f'https://api.legiscan.com/?key={api_key}&op=getSearch&state={state}&query={term.replace(" ", "+")}'
    assemble_params = {
        'key': api_key,
        'op': 'getSearch',
        'state': state,
        'query': term,
    }
    return requests.get(short_url, params=assemble_params).text

json.loads(do_search('ME', '577')) # needs to be exact or the search can't find it :/

In [ ]:
from bs4 import BeautifulSoup as Soup
from itertools import islice
from nltk import download as nltk_download, pos_tag, word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import string

nltk_download('punkt')
nltk_download('averaged_perceptron_tagger')
nltk_download('wordnet')

lem = WordNetLemmatizer()

def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
    
def extract_html_file(file_path: str):
    soup = None
    with open(file_path, 'r') as f:
        soup = Soup(f, 'html.parser')

    for script in soup(["script", "style"]):
        script.extract()    # rip it out

    text = soup.get_text()
    tokens = (token for token in word_tokenize(text) if token not in string.punctuation)
    return list(tokens)

[(orig, lem) for orig, lem in ((word, lem.lemmatize(word, get_wordnet_pos(pos))) for word, pos in pos_tag(extract_html_file('bills/FL_S0254.html'))) if orig != lem]

In [ ]:
from bs4 import BeautifulSoup as Soup
from itertools import islice
import nltk
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from pprint import pprint
import string

nltk.download('punkt')

def extract_html_file(file_path: str):
    soup = None
    with open(file_path, 'r') as f:
        soup = Soup(f, 'html.parser')

    for script in soup(["script", "style"]):
        script.extract()    # rip it out

    text = soup.get_text()
    tokens = (token for token in word_tokenize(text) if token not in string.punctuation)
    return tokens
    # break into lines and remove leading and trailing space on each
    #lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
    #chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    #text = "\n".join(chunk for chunk in chunks if chunk)

    #return text

ps = PorterStemmer()

pprint([(word, ps.stem(word)) for word in islice(extract_html_file('bills/FL_S0254.html'), 100)])


In [ ]:
# import base64
import json
from pprint import pprint

filename = 'tmp/bill_meta_TX_HB1532.json'

result = None
with open(filename, 'r') as f:
    result = json.load(f)['bill']

extra = result['texts'][0].copy()
extra['date'] = '2023-01-01'
result['texts'].append(extra)
# pprint(result['texts'])
s = sorted(result['texts'], key=lambda x: x['date'], reverse=True)
doc_id = result['texts'][0]['doc_id']
doc_id

In [ ]:
from legiscan import legiscan_api
import pandas as pd

raw = pd.read_json('tracktranslegislation.json')
sample = raw.sample(n=3, random_state=1234)

@legiscan_api
def get_bill_meta(legiscan_bill_id: str, api_key: str):
    assembled_url = f'https://api.legiscan.com/?key={api_key}&op=getBill&id={legiscan_bill_id}'
    resp = requests.get(assembled_url)

    if resp.ok:
        parsed = json.loads(resp.text)
        if parsed['status'] == 'ERROR':
            print(parsed['alert']['message'])
            return None
        return resp
    else:
        print(resp.status_code)
        return None
    
for idx, row in sample.iterrows():
    local_filename = os.path.join(
        'tmp',
        '_'.join([
            'bill',
            'meta',
            row["state"],
            *row["billId"].split(' '),
        ])
    ) + '.json'
    
    if os.path.exists(local_filename):
        print(f'skipping {local_filename}')
        continue
    
    resp = None
    resp = get_bill_meta(row['legiscanId'])
    if not resp:
        print(f'Could not download {local_filename}')
        continue
    
    print(f'got {local_filename}')
    with open(local_filename, 'wb') as f:
        f.write(resp.content)


In [ ]:
raw = pd.read_json('tracktranslegislation.json')
sample = raw.sample(n=3, random_state=1234)
sample

# 7 has wrong legiscanId, should be 2721785 i think
# 44 also has wrong legiscanId

In [ ]:
import base64
import json
from pprint import pprint

filename = 'bills/TX_HB1532'

result = None
with open(filename, 'r') as f:
    result = json.load(f)['text']

doc = result['doc']
extension = result['mime'].split('/')[-1]
new_filename = '.'.join([filename, extension])

with open(new_filename, 'wb') as f:
    f.write(base64.b64decode(doc))


In [ ]:
import json
from legiscan import legiscan_api
import os
import pandas as pd
from pprint import pprint
import requests

@legiscan_api
def get_bill_text(legiscan_bill_id: str, api_key: str):
    # https://api.legiscan.com/?key=5f61f50916512f9f21500f38877c22f7&op=getBillText&id=2736883
    assembled_url = f'https://api.legiscan.com/?key={api_key}&op=getBillText&id={legiscan_bill_id}'
    resp = requests.get(assembled_url)

    if resp.ok:
        parsed = json.loads(resp.text)
        if parsed['status'] == 'ERROR':
            print(parsed['alert']['message'])
            return None
        return resp
    else:
        print(resp.status_code)
        return None

raw = pd.read_json('tracktranslegislation.json')
sample = raw.sample(n=3, random_state=1234)

for idx, row in sample.iterrows():
    local_filename = os.path.join(
        'bills',
        '_'.join([
            row["state"],
            *row["billId"].split(' '),
        ])
    )
    
    if os.path.exists(local_filename):
        print(f'skipping {local_filename}')
    
    resp = get_bill_text(row['legiscanId'])
    if not resp:
        print(f'Could not download {local_filename}')
        continue
    
    with open(local_filename, 'wb') as f:
        f.write(resp.content)


In [ ]:
from functools import wraps
from legiscan import legiscan_api
import os

@legiscan_api
def sample_api_action(api_key: str):
    print(f'api key is {api_key}')

@legiscan_api
def get_bill_text(legiscan_bill_id: str, api_key: str):
    # https://api.legiscan.com/?key=5f61f50916512f9f21500f38877c22f7&op=getBillText&id=2736883
    assembled_url = f'https://api.legiscan.com/?key={api_key}&op=getBillText&id={legiscan_bill_id}'
    print(assembled_url)
    
sample_api_action(api_key='foo')


get_bill_text('12345')

In [ ]:
from bs4 import BeautifulSoup
import requests
import urllib.parse

host = 'https://www.house.mo.gov'
# url = f'{host}/BillContent.aspx?bill=HB1258&year=2023&code=R'
url = urllib.parse.urljoin(host, 'BillContent.aspx?bill=HB1258&year=2023&code=R')
page = requests.get(url)

# print(page.text)
soup = BeautifulSoup(page.content)
urllib.parse.urljoin(
    host, 
    soup.find_all(class_='textType')[0].find('a')['href'],
)


In [ ]:
import pandas as pd
import requests
from typing import Optional
import urllib.parse

def test_url(url: str, parent_id: str, anchor_index: int):
    page = requests.get(url)
    soup = BeautifulSoup(page.content)
    container = soup.find(id=parent_id)
    if not container:
        return None
    
    try:
        return container.find_all('a')[anchor_index]['href']
    except:
        return None

def prepare_url(relative_url: Optional[str]):
    if not relative_url:
        return 'NO RESULT'
    
    return urllib.parse.urljoin('https://legiscan.com/', relative_url)

def process_as_bill(frame) -> Optional[str]:
    return process_bill_link(frame['billLink'])

def process_bill_link(url: str) -> Optional[str]:
    print(url)
    try:
        page = requests.get(url)
        soup = BeautifulSoup(page.content)
        container = soup.find(id='bill-last-action')
        anchors = container.find_all('a')
        href = anchors[-1]['href']
        return urllib.parse.urljoin('https://legiscan.com', href)
    except:
        return None

def process_as_text(frame) -> Optional[str]:
    state = frame['state']
    bill_id = frame['billId'].replace(' ', '')
    year = row['billLink'].split('/')[-1] # don't trust statusDate
    text_link = f'https://legiscan.com/{state}/text/{bill_id}/{year}'
    return process_text_link(text_link)

def process_text_link(url: str) -> Optional[str]:
    print(url)
    try:
        page = requests.get(url)
        soup = BeautifulSoup(page.content)
        container = soup.find(id='gaits-wrapper')
        href = container.find_all('a')[-1]['href']
        return urllib.parse.urljoin('https://legiscan.com', href)
    except:
        return None

def process_as_draft(frame) -> Optional[str]:
    state = frame['state']
    bill_id = frame['billId'].replace(' ', '')
    year = row['billLink'].split('/')[-1] # don't trust statusDate
    text_link = f'https://legiscan.com/{state}/drafts/{bill_id}/{year}'
    return process_text_link(text_link)
    
def process_draft_link(url: str) -> Optional[str]:
    print(url)
    try:
        page = requests.get(url)
        soup = BeautifulSoup(page.content)
        container = soup.find(id='gaits-wrapper')
        href = container.find_all('a')[-1]['href']
        return urllib.parse.urljoin('https://legiscan.com', href)
    except:
        return None

raw = pd.read_json('tracktranslegislation.json')
sample = raw.copy()
# sample = raw.sample(n=20, random_state=2)
# sample = raw.loc[raw.state == 'AR']
# sample = raw.loc[0:20]
# print(sample)

for idx, row in sample.iterrows():
    continue
    bill_id = ' '.join([row['state'], row['billId']])
#    year = row['billLink'].split('/')[-1]
    
#    bill_link = row['billLink']
#    draft_link = f'https://legiscan.com/{row["state"]}/drafts/{row["billId"].replace(" ", "")}/{year}' # https://legiscan.com/AZ/drafts/HB2517/2023
#    text_link = f'https://legiscan.com/{row["state"]}/text/{row["billId"].replace(" ", "")}/{year}'
#    comments_link = f'https://legiscan.com/{row["state"]}/comments/{row["billId"].replace(" ", "")}/{year}'
    
    searches = [
        process_as_bill,
        process_as_text,
    ]

    print(f'{bill_id}')
    for search in searches:
        print(search(row))

    print()


In [ ]:
# https://legiscan.com/TX/comments/HB1029/2023

from bs4 import BeautifulSoup
from pprint import pprint
import requests
from typing import Optional
import urllib.parse

def process_as_bill(frame) -> Optional[str]:
    return process_bill_link(frame['billLink'])

def process_bill_link(url: str) -> Optional[str]:
    try:
        page = requests.get(url)
        soup = BeautifulSoup(page.content)
        container = soup.find(id='bill-last-action')
        anchors = container.find_all('a')
        href = anchors[-1]['href']
        return urllib.parse.urljoin('https://legiscan.com', href)
    except:
        return None

def process_as_text(frame) -> Optional[str]:
    state = frame['state']
    bill_id = frame['billId'].replace(' ', '')
    year = row['billLink'].split('/')[-1] # don't trust statusDate
    text_link = f'https://legiscan.com/{state}/text/{bill_id}/{year}'
    return process_text_link(text_link)

def process_text_link(url: str) -> Optional[str]:
    try:
        page = requests.get(url)
        soup = BeautifulSoup(page.content)
        container = soup.find(id='gaits-wrapper')
        href = container.find_all('a')[-1]['href']
        return urllib.parse.urljoin('https://legiscan.com', href)
    except:
        return None

def process_as_draft(frame) -> Optional[str]:
    state = frame['state']
    bill_id = frame['billId'].replace(' ', '')
    year = row['billLink'].split('/')[-1] # don't trust statusDate
    text_link = f'https://legiscan.com/{state}/drafts/{bill_id}/{year}'
    return process_text_link(text_link)
    
def process_draft_link(url: str) -> Optional[str]:
    try:
        page = requests.get(url)
        soup = BeautifulSoup(page.content)
        container = soup.find(id='gaits-wrapper')
        href = container.find_all('a')[-1]['href']
        return urllib.parse.urljoin('https://legiscan.com', href)
    except:
        return None

    
#text_link = 'https://legiscan.com/TX/text/HB3147/2023'
#print(text_link)
#print(process_text_link(text_link))

#bill_link = 'https://legiscan.com/TX/bill/HB976/2023'
#print(bill_link)
#print(process_bill_link(bill_link))
#print(process_as_bill(sample.loc[303]))

print(process_as_bill(raw.loc[251]))
print(process_as_text(raw.loc[251]))
print(process_as_draft(raw.loc[251]))

#host = 'https://www.house.mo.gov'
# url = f'{host}/BillContent.aspx?bill=HB1258&year=2023&code=R'
#url = 'https://legiscan.com/SD/text/HB1080/2023' #urllib.parse.urljoin(host, 'BillContent.aspx?bill=HB1258&year=2023&code=R')
#page = requests.get(url)

# print(page.text)
#soup = BeautifulSoup(page.content)
#container = soup.find(id='gaits-wrapper')
#pprint(container.find_all('a'))
#urllib.parse.urljoin(
#    host, 
#    soup.find_all(class_='textType')[0].find('a')['href'],
#)